# 確認用

###  np.isscalarを試す

In [4]:
import numpy as np

print(np.isscalar(np.float64(1.0)))
print(np.isscalar(1.0))
print(np.isscalar(np.array(1.0)))
print(np.isscalar(np.array([1, 2, 3])))

True
True
False
False


### 可変長引数

In [1]:
def f(*x):
    print(x)

f(1)
f(1, 2, 3)
f(1, 2, 3, 4, 5, 6)

(1,)
(1, 2, 3)
(1, 2, 3, 4, 5, 6)


### ダミーの関数でgenerationの取り出し

In [15]:
import numpy as np
import heapq as hq

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supportted'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None # 変数にとって生みの親である関数
        self.generation = 0 # 正しい逆伝播のための世代
    
    def set_creator(self, func):
        '''
        creatorを指定しない変数も存在するので，set_creatorという関数が必要
        '''
        self.creator = func
        # input側から，output側へ set_creator 関数を実行するので，自分の関数の generation に 1 を加える
        self.generation = func.generation + 1

    def backward(self):
        if self.grad is None: # grad の初期化に使用する
            self.grad = np.ones_like(self.data) # self.data と型も同じになる

        funcs = [self.creator] # 次の関数を見つけるたびにここに追加される
        while funcs:
            f = funcs.pop()

            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)

            if not isinstance(gxs, tuple): # 単にgxに要素のみ（スカラ）が入っている場合でも，タプルに変換可能
                gxs = (gxs,) # なぜタプルにするんだっけ？
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # それまでの勾配に，別の勾配を足し合わせる

                if x.creator is not None:
                    funcs.append(x.creator)

    def cleargrad(self):
        self.grad = None

class Function:
    def __call__(self, *inputs): # アスタリスクは可変長引数を表す
        '''
        入力が複数ある場合に対応させる
        '''
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # アスタリスクをつけてアンパッキングする
        if not (isinstance(ys, tuple)):
            ys = (ys,) # ys をタプルにする
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) # 入力の中の，最も大きい generation に合わせる
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # リストの要素が1つの時は，最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other): # 関数同士の比較に必要？
        return self.generation < other.generation

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()
    
generations = [2, 0, 1, 4, 2]
alphabets = ['d', 'x', 'b', 'q', 'r']
funcs = []

for g, a in zip(generations, alphabets):
    f = Function()
    f.generation = g
    hq.heappush(funcs, (-1*f.generation, f))

# print(hq.heappop(funcs))
_, f = hq.heappop(funcs)
print(f.generation)

# print([f.generation for f in funcs])

# funcs.sort(key=lambda x: x.generation)
# print([f.generation for f in funcs])

# f = funcs.pop()
# print(f.generation)

4


In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supportted'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None # 変数にとって生みの親である関数
        self.generation = 0 # 正しい逆伝播のための世代
    
    def set_creator(self, func):
        '''
        creatorを指定しない変数も存在するので，set_creatorという関数が必要
        '''
        self.creator = func
        # input側から，output側へ set_creator 関数を実行するので，自分の関数の generation に 1 を加える
        self.generation = func.generation + 1

    def backward(self):
        if self.grad is None: # grad の初期化に使用する
            self.grad = np.ones_like(self.data) # self.data と型も同じになる

        funcs = []
        seen_set = set() # 集合を使うことで，同じ関数を重複して追加することを避ける

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)
        
        add_func(self.creator)

        while funcs:
            f = funcs.pop()

            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)

            if not isinstance(gxs, tuple): # 単にgxに要素のみ（スカラ）が入っている場合でも，タプルに変換可能
                gxs = (gxs,) # なぜタプルにするんだっけ？
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # それまでの勾配に，別の勾配を足し合わせる

                if x.creator is not None:
                    add_func(x.creator)

    def cleargrad(self):
        self.grad = None

class Function:
    def __call__(self, *inputs): # アスタリスクは可変長引数を表す
        '''
        入力が複数ある場合に対応させる
        '''
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # アスタリスクをつけてアンパッキングする
        if not (isinstance(ys, tuple)):
            ys = (ys,) # ys をタプルにする
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) # 入力の中の，最も大きい generation に合わせる
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # リストの要素が1つの時は，最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

## メモリ管理

In [ ]:
class obj:
    pass

def f(x):
    print(x)

a = obj() # 参照カウント 1 
f(a) # 関数内では，参照カウント 2
# 関数を抜けて，参照カウント 1
a = None # 参照カウント 0（Noneを代入しているから？）

# オブジェクト同士の参照の連鎖
a = obj()
b = obj()
c = obj()
a.b = b
b.c = c
a = b = c = None

### 参照カウント
- 連鎖的に参照カウントが0になることで，ユーザから参照されていないオブジェクトはまとめて消去されることになる

→ 循環参照の場合は，参照カウントが0にならず，メモリから解放されない

### Grabage Collection (GC)
メモリが不足したタイミングなどで Python インタプリタによって自動的に呼び出される．
- GC によって循環参照を解消できるが，GCによってメモリの開放を先送りすることは，プログラム全体のメモリ使用量を増加させる原因になる

In [2]:
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

print(b)
print(b()) # 参照先のデータにアクセスするには，b()とする

a = None
print(b) # b が dead になる

<weakref at 0x7f4970e419a0; to 'numpy.ndarray' at 0x7f494ea53990>
[1 2 3]
<weakref at 0x7f4970e419a0; dead>


In [1]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

import numpy as np
from dezero import Variable
from dezero.utils import get_dot_graph

x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
y = x0 + x1

x0.name = 'x0'
x1.name = 'x1'
y.name = 'y'

txt = get_dot_graph(y, verbose=False)
print(txt)

with open('sample.dot', 'w') as o:
    o.write(txt)

ModuleNotFoundError: No module named 'dezero'